In [1]:
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langserve import RemoteRunnable
import bs4
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain import hub
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.vectorstores import VectorStoreRetriever
from langchain_core.documents.base import Document
from langchain.agents import AgentExecutor
from langchain.tools.retriever import create_retriever_tool
import os

In [2]:
from dotenv import load_dotenv

# 환경변수 로드 (.env)
load_dotenv()

True

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    base_url = "http://sionic.chat:8001/v1",
    api_key = "934c4bbc-c384-4bea-af82-1450d7f8128d",
    model="xionic-ko-llama-3-70b",
    temperature=0
)

In [4]:
from langchain.tools.retriever import create_retriever_tool

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs = {'device': 'cpu'}, # 모델이 CPU에서 실행되도록 설정. GPU를 사용할 수 있는 환경이라면 'cuda'로 설정할 수도 있음
    encode_kwargs = {'normalize_embeddings': True}, # 임베딩 정규화. 모든 벡터가 같은 범위의 값을 갖도록 함. 유사도 계산 시 일관성을 높여줌
)

# 로컬 DB 불러오기
MY_NEWS_INDEX = "MY_NEWS_INDEX"
vectorstore1 = FAISS.load_local(MY_NEWS_INDEX, 
                               embeddings, 
                               allow_dangerous_deserialization=True)
retriever1 = vectorstore1.as_retriever(search_type="similarity", search_kwargs={"k": 3}) # 유사도 높은 3문장 추출
MY_PDF_INDEX = "MY_PDF_INDEX"
vectorstore2 = FAISS.load_local(MY_PDF_INDEX, 
                               embeddings, 
                               allow_dangerous_deserialization=True)
retriever2 = vectorstore2.as_retriever(search_type="similarity", search_kwargs={"k": 3}) # 유사도 높은 3문장 추출

retriever_tool1 = create_retriever_tool(
    retriever1,
    name="saved_news_search",
    description="엔비디아, 퍼플렉시티, 라마3 관련 정보를 검색한다. 엔비디아, 퍼플렉시티, 라마3 관련 정보는 이 도구를 사용해야 한다",
)

retriever_tool2 = create_retriever_tool(
    retriever2,
    name="pdf_search",
    description="생성형 AI 신기술 도입에 따른 선거 규제 연구 관련 정보를 검색한다. 생성형 AI 신기술 도입에 따른 선거 규제 연구 관련 정보는 이 도구를 사용해야 한다",
)

tools = [retriever_tool1, retriever_tool2]
tools

/Users/kwon0koang/Library/Caches/pypoetry/virtualenvs/test-langchain-PJ4Uqdqm-py3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/Users/kwon0koang/Library/Caches/pypoetry/virtualenvs/test-langchain-PJ4Uqdqm-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Tool(name='saved_news_search', description='엔비디아, 퍼플렉시티, 라마3 관련 정보를 검색한다. 엔비디아, 퍼플렉시티, 라마3 관련 정보는 이 도구를 사용해야 한다', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x10646a5c0>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x2bab41bb0>, search_kwargs={'k': 3}), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x10646aac0>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x2bab41bb0>, search_kwargs={'k': 3}), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n')),
 Tool(name='pdf_search', description='생성형 AI 신기술 도입에 따른 선거 규제 연구 관련 정보를

In [5]:
json_prompt = hub.pull("teddynote/react-chat-json-korean")
json_prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'teddynote', 'lc_hub_repo': 'react-chat-json-korean', 'lc_hub_commit_hash': '246b7852986ad9370b8d41310461e4e52a947ea1b67bf034722f4447a4caa064'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Assistant is a large language model trained by OpenAI.\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate hum

In [8]:
from langchain.agents import create_json_chat_agent
from langchain.agents import AgentExecutor

agent = create_json_chat_agent(llm, tools, json_prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    return_intermediate_steps=True,
)

In [10]:
response = agent_executor.invoke(
    # {"input": "퍼플렉시티가 투자받은 금액?"}
    {"input": "생성형 AI 도입에 따른 규제 연구 책임자는?"}
) 
response["output"]



> Entering new AgentExecutor chain...
```
{
    "action": "pdf_search",
    "action_input": "생성형 AI 도입에 따른 규제 연구 책임자는"
}
```ii
【2023년도 중앙선거관리위원회 정책연구용역 보고서】
『생성형 AI 신기술 도입에 따른 
선거 규제 연구』
연구책임자 : 
김
 주
 희(국 립 부 경 대 학 교)
공동연구자 :
차
 재
 권(국 립 부 경 대 학 교)
김
 현
 정(동 아 대 학 교)
조
 성
 복(국 민 대 학 교)
연구보조원 : 
박
 서
 현(국 립 부 경 대 학 교)
권
 수
 민(국 립 부 경 대 학 교)
본 연구보고서는 2023년도 중앙선거관리위원회 정책연구용역 과제로서 연구내용은 중앙
선거관리위원회의 공식 견해가 아님.

15
나. 생성형 AI 규제 유형화
○본 연구에서는 주요국의 생성형 AI 규제 동향을 ‘AI 관련 입법 목적’ 
및 ‘AI에 대한 규제는 규제원칙 및 방식’에 따라 아래와 같이 분류한다.
○다수 국가들이 AI 규제를 준비하고 있는 가운데, 미국과 EU는 AI에 대
한 규제 입법에 앞장서 왔다. 미국과 EU의 규제 내용 및 동향 대한 자
세한 내용은 4장에서 다룰 것이다.
- 2019년 트럼프 전 대통령이 행정명령 제13859호에서 “미국 인공지
능 구상”을 공표한 이후, 미국은 AI 관련 규제를 입법하였다. 본 연
구에서는 AI 연구개발과 훈련과 관련한 법률인 「생성적 적대 신경
망 출력물 확인법」, 「국가 인공지능 구상법 2020」을 분석한다. 또
한 연방정부가 제정한 AI 사용에 관한 행정명령 제13960호, 「정부인
공지능법 2020」, 「조달인력의 인공지능 역량강화법」, 「미국인공
지능진흥법」 등과 더불어 최근 논의되고 있는 생성형 AI 기술 규제

- (제8조) 생성형 AI 기술 연구·개발 과정에서 데이터 라벨링을 수행하
는 경우 서비스 제공자는 이 방법의 요구사항에 따라 △명확하고 구체
적이며 실행 가능한 라벨링 규칙을 제정

'생성형 AI 도입에 따른 규제 연구 책임자는 김주희(국립부경대학교)입니다.'